In [1]:
from transformers import pipeline

C:\Users\serma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "PlanTL-GOB-ES/bsc-bio-ehr-es"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)



C:\Users\serma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/bsc-bio-ehr-es and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
import os

# Cargar el archivo CSV
data = pd.read_csv('datasets2/data.csv')
output_dir = "./results"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# Convertir el DataFrame a un Dataset de Hugging Face
dataset = Dataset.from_pandas(data)

# Dividir el dataset en entrenamiento y prueba
dataset = dataset.train_test_split(test_size=0.2)




# bsc-bio-ehr-es

### Tokenizar los datos

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/bsc-bio-ehr-es")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 20/20 [00:00<00:00, 1052.46 examples/s]


### Configurar y entrenar el modelo

In [5]:
import os
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
from datasets import load_metric



# 3. Configurar y entrenar el modelo
model = AutoModelForSequenceClassification.from_pretrained("PlanTL-GOB-ES/bsc-bio-ehr-es", num_labels=2)

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir=None,  # Desactivar TensorBoard
    report_to=[],  # Desactivar el reporte a TensorBoard
)

# Definir la función de evaluación
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/bsc-bio-ehr-es and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\serma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                              
 33%|███▎      | 5/15 [02:50<05:05, 30.54s/it]

{'eval_loss': 0.6217153668403625, 'eval_accuracy': 0.7, 'eval_runtime': 6.5302, 'eval_samples_per_second': 3.063, 'eval_steps_per_second': 0.306, 'epoch': 1.0}


                                               
 67%|██████▋   | 10/15 [05:23<02:39, 31.86s/it]

{'eval_loss': 0.5629316568374634, 'eval_accuracy': 0.9, 'eval_runtime': 6.4143, 'eval_samples_per_second': 3.118, 'eval_steps_per_second': 0.312, 'epoch': 2.0}


                                               
100%|██████████| 15/15 [07:58<00:00, 31.88s/it]

{'eval_loss': 0.5432060956954956, 'eval_accuracy': 0.9, 'eval_runtime': 6.2852, 'eval_samples_per_second': 3.182, 'eval_steps_per_second': 0.318, 'epoch': 3.0}
{'train_runtime': 478.1676, 'train_samples_per_second': 0.502, 'train_steps_per_second': 0.031, 'train_loss': 0.6241563161214193, 'epoch': 3.0}


TrainOutput(global_step=15, training_loss=0.6241563161214193, metrics={'train_runtime': 478.1676, 'train_samples_per_second': 0.502, 'train_steps_per_second': 0.031, 'total_flos': 63146653286400.0, 'train_loss': 0.6241563161214193, 'epoch': 3.0})

In [6]:
results = trainer.evaluate()

print(results)

100%|██████████| 2/2 [00:01<00:00,  1.72it/s]

{'eval_loss': 0.5432060956954956, 'eval_accuracy': 0.9, 'eval_runtime': 6.1422, 'eval_samples_per_second': 3.256, 'eval_steps_per_second': 0.326, 'epoch': 3.0}


# PlanTL-GOB-ES/roberta-base-biomedical-clinical-es

In [8]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("BSC-TeMU/roberta-base-biomedical-es")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
# 1. Cargar el archivo CSV y preparar el dataset
data = pd.read_csv('datasets2/data.csv')
dataset = Dataset.from_pandas(data)
dataset = dataset.train_test_split(test_size=0.2)

# 2. Tokenizar los datos
tokenizer = AutoTokenizer.from_pretrained("BSC-TeMU/roberta-base-biomedical-es")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Asegurarse de que el directorio de resultados existe
output_dir = "./results"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 3. Configurar y entrenar el modelo
model = AutoModelForSequenceClassification.from_pretrained("BSC-TeMU/roberta-base-biomedical-es", num_labels=2)

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Ajustar a un tamaño más razonable
    per_device_eval_batch_size=16,  # Ajustar a un tamaño más razonable
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir=None,  # Desactivar TensorBoard si no es necesario
    report_to=[],  # Desactivar el reporte a TensorBoard
)

# Definir la función de evaluación
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

trainer.train()

# 4. Evaluar el modelo
results = trainer.evaluate()

print(results)




Map: 100%|██████████| 80/80 [00:00<00:00, 721.36 examples/s]

Map: 100%|██████████| 20/20 [00:00<00:00, 788.32 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-biomedical-es and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\serma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 33%|███▎      | 5/15 [02:52<04:59, 29.91s/it]

                                              
                                   

 33%|███▎      | 5/15 [02:59<04:5

{'eval_loss': 0.7026912569999695, 'eval_accuracy': 0.45, 'eval_runtime': 6.4425, 'eval_samples_per_second': 3.104, 'eval_steps_per_second': 0.31, 'epoch': 1.0}


 67%|██████▋   | 10/15 [05:22<02:23, 28.72s/it]

                                               
                                   

 67%|██████▋   | 10/15 [05:29<02:23, 28.72s/it]



{'eval_loss': 0.6966423988342285, 'eval_accuracy': 0.7, 'eval_runtime': 6.8637, 'eval_samples_per_second': 2.914, 'eval_steps_per_second': 0.291, 'epoch': 2.0}


100%|██████████| 15/15 [07:40<00:00, 25.88s/it]

                                               
                                   

100%|██████████| 15/15 [07:47<00:00, 25.88s/it]

                                               
100%|██████████| 15/15 [07:47<00:00, 31.16s/it]


{'eval_loss': 0.7046510577201843, 'eval_accuracy': 0.55, 'eval_runtime': 6.3032, 'eval_samples_per_second': 3.173, 'eval_steps_per_second': 0.317, 'epoch': 3.0}
{'train_runtime': 467.3344, 'train_samples_per_second': 0.514, 'train_steps_per_second': 0.032, 'train_loss': 0.641140874226888, 'epoch': 3.0}


100%|██████████| 2/2 [00:01<00:00,  1.79it/s]

{'eval_loss': 0.7046510577201843, 'eval_accuracy': 0.55, 'eval_runtime': 5.8667, 'eval_samples_per_second': 3.409, 'eval_steps_per_second': 0.341, 'epoch': 3.0}


In [ ]:
from transformers import RobertaTokenizer

# Cargar el tokenizador utilizando los archivos relevantes
tokenizer = RobertaTokenizer.from_pretrained(
    'path/to/tokenizer/directory'
)
